# Load the Pretrained Model and the dataset
We use bilstm as the model and chnsenticorp as the dataset for example. More models can be found in [PaddleNLP Model Zoo](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/transformers.html#transformer).

BiLSTM does not have a tokenizer so we use JiebaTokenizer.

Obviously, PaddleNLP is needed to run this notebook, which is easy to install:
```bash
pip install setuptools_scm 
pip install --upgrade paddlenlp==2.1 
```

In [1]:
# !wget https://paddlenlp.bj.bcebos.com/data/senta_word_dict.txt -P assets/
from paddlenlp.data import JiebaTokenizer, Pad, Stack, Tuple, Vocab

VOCAB_PATH = "assets/senta_word_dict.txt"

vocab = Vocab.load_vocabulary(VOCAB_PATH, unk_token='[UNK]', pad_token='[PAD]')

vocab_size = len(vocab)
num_classes = 2
pad_token_id = vocab.to_indices('[PAD]')

In [2]:
from paddlenlp.datasets import load_dataset
DATASET_NAME = 'chnsenticorp'
train_ds, dev_ds, test_ds = load_dataset(
    DATASET_NAME, splits=["train", "dev", "test"]
)

In [3]:
import paddle
import paddlenlp
import paddle.nn as nn
import paddle.nn.functional as F

MODEL_NAME = 'bilstm'

class LSTMModel(nn.Layer):
    def __init__(self,
                 vocab_size,
                 num_classes,
                 emb_dim=128,
                 padding_idx=0,
                 lstm_hidden_size=198,
                 direction='forward',
                 lstm_layers=1,
                 dropout_rate=0.0,
                 pooling_type=None,
                 fc_hidden_size=96):
        super().__init__()

        # 首先将输入word id 查表后映射成 word embedding
        self.embedder = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            padding_idx=padding_idx)

        # 将word embedding经过LSTMEncoder变换到文本语义表征空间中
        self.lstm_encoder = paddlenlp.seq2vec.LSTMEncoder(
            emb_dim,
            lstm_hidden_size,
            num_layers=lstm_layers,
            direction=direction,
            dropout=dropout_rate,
            pooling_type=pooling_type)

        # LSTMEncoder.get_output_dim()方法可以获取经过encoder之后的文本表示hidden_size
        self.fc = nn.Linear(self.lstm_encoder.get_output_dim(), fc_hidden_size)

        # 最后的分类器
        self.output_layer = nn.Linear(fc_hidden_size, num_classes)

    def forward(self, text, seq_len):
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)

        # Shape: (batch_size, num_tokens, num_directions*lstm_hidden_size)
        # num_directions = 2 if direction is 'bidirectional' else 1
        text_repr = self.lstm_encoder(embedded_text, sequence_length=seq_len)


        # Shape: (batch_size, fc_hidden_size)
        fc_out = paddle.tanh(self.fc(text_repr))

        # Shape: (batch_size, num_classes)
        logits = self.output_layer(fc_out)
        return logits

model_ = LSTMModel(
        len(vocab),
        2,
        direction='bidirectional',
        padding_idx=vocab['[PAD]'])

model = paddle.Model(model_)

W1104 18:54:26.319262 24749 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.2
W1104 18:54:26.324112 24749 device_context.cc:422] device: 0, cuDNN Version: 7.6.


# Prepare the Model

## Train the model

In [4]:
import numpy as np
from functools import partial
from assets.utils import create_dataloader

def convert_example(example, tokenizer, is_test=False):
    input_ids = tokenizer.encode(example["text"])
    valid_length = np.array(len(input_ids), dtype='int64')
    input_ids = np.array(input_ids, dtype='int64')

    if not is_test:
        label = np.array(example["label"], dtype="int64")
        return input_ids, valid_length, label
    else:
        qid = np.array(example["qid"], dtype="int64")
        return input_ids, valid_length, qid

# Reads data and generates mini-batches.
tokenizer = JiebaTokenizer(vocab)
trans_fn = partial(convert_example, tokenizer=tokenizer, is_test=False)

batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=vocab.token_to_idx.get('[PAD]', 0)),  # input_ids
    Stack(dtype="int64"),  # seq len
    Stack(dtype="int64")  # label
): [data for data in fn(samples)]
                        
train_loader = create_dataloader(
    train_ds,
    trans_fn=trans_fn,
    batch_size=128,
    mode='train',
    batchify_fn=batchify_fn)
                        
dev_loader = create_dataloader(
    dev_ds,
    trans_fn=trans_fn,
    batch_size=128,
    mode='validation',
    batchify_fn=batchify_fn)

optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(), learning_rate=5e-4)

# Defines loss and metric.
criterion = paddle.nn.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

model.prepare(optimizer, criterion, metric)

model.fit(
    train_loader,
    dev_loader,
    epochs=3,
    save_dir=f'assets/{DATASET_NAME}-{MODEL_NAME}',
    verbose=1
)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/3
step 75/75 [==============================] - loss: 0.3585 - acc: 0.6821 - 125ms/step          
save checkpoint at /root/codespace/InterpretDL-master/tutorials/assets/chnsenticorp-bilstm/0
Eval begin...
step 10/10 [==============================] - loss: 0.3471 - acc: 0.8642 - 75ms/step
Eval samples: 1200
Epoch 2/3
step 75/75 [==============================] - loss: 0.1716 - acc: 0.9304 - 113ms/step          
save checkpoint at /root/codespace/InterpretDL-master/tutorials/assets/chnsenticorp-bilstm/1
Eval begin...
step 10/10 [==============================] - loss: 0.4046 - acc: 0.8908 - 72ms/step
Eval samples: 1200
Epoch 3/3
step 75/75 [==============================] - loss: 0.0475 - acc: 0.9811 - 111ms/step          
save checkpoint at /root/codespace/InterpretDL-master/tutorials/assets/chnsenticorp-bilstm/2
Eval begin...
step 10/10 [==============================]

## Or Load the trained model

In [4]:
# Load the trained model.
state_dict = paddle.load(f'assets/{DATASET_NAME}-{MODEL_NAME}/final.pdparams')
model_.set_dict(state_dict)

# See the prediction results

In [5]:
from assets.utils import predict

data = [
    {"text":'这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'},
    {"text":'怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'},
    {"text":'作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'},
]

label_map = {0: 'negative', 1: 'positive'}

batch_size = 32

# results = predict(
#     model, data, tokenizer, label_map, batch_size=batch_size)

# for idx, text in enumerate(data):
#     print('Data: {} \t Lable: {}'.format(text, results[idx]))

# Prepare for Interpretations

In [6]:
import interpretdl as it
import numpy as np
from assets.utils import convert_example, aggregate_subwords_and_importances
from paddlenlp.data import Stack, Tuple, Pad
from interpretdl.data_processor.visualizer import VisualizationTextRecord, visualize_text

import jieba
def preprocess_fn(text):
    texts = []
    seq_lens = []
    tokens = " ".join(jieba.cut(text)).split(' ')
    ids = []
    unk_id = vocab.token_to_idx.get('[UNK]', None)
    for token in tokens:
        wid = vocab.token_to_idx.get(token, unk_id)
        if wid:
            ids.append(wid)
    texts.append(ids)
    seq_lens.append(len(ids))

    pad_token_id = 0
    max_seq_len = max(seq_lens)

    texts = paddle.to_tensor(texts)
    seq_lens = paddle.to_tensor(seq_lens)
    return texts, seq_lens

InterpretDL Current Version: 0.3.0rc


## IG Interpreter

In [8]:
ig = it.IntGradNLPInterpreter(model_, True)

true_labels = [0, 0, 1]
recs = []
for review in data:
    text = review['text']

    pred_labels, pred_probs, avg_gradients = ig.interpret(
        preprocess_fn(text),
        steps=10,
        embedding_name='embedder',
        return_pred=True)

    for i in range(avg_gradients.shape[0]):
        subwords = " ".join(jieba.cut(text)).split(' ')
        subword_importances = avg_gradients[i]
        words, word_importances = aggregate_subwords_and_importances(subwords, subword_importances)
        word_importances = np.array(word_importances) / np.linalg.norm(
            word_importances)

        pred_label = pred_labels[i]
        pred_prob = pred_probs[i, pred_label]
        true_label = true_labels[i]
        interp_class = pred_label

        if interp_class == 0:
            word_importances = -word_importances
        recs.append(
            VisualizationTextRecord(words, word_importances, true_label,
                                    pred_label, pred_prob, interp_class)
        )

visualize_text(recs)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.803 seconds.
Prefix dict has been built successfully.


## LIME Interpreter

In [9]:
true_labels = [0, 0, 1]
recs = []

lime = it.LIMENLPInterpreter(model_)
for i, review in enumerate(data):
    review = review['text']
    pred_class, pred_prob, lime_weights = lime.interpret(
        review,
        preprocess_fn,
        num_samples=1000,
        batch_size=10,
        unk_id=vocab['[UNK]'],
        pad_id=0,
        return_pred=True)

    # subwords
    subwords = " ".join(jieba.cut(review)).split(' ')
    interp_class = list(lime_weights.keys())[0]
    weights = lime_weights[interp_class][1 : -1]
    subword_importances = [t[1] for t in lime_weights[interp_class][1 : -1]]
    
    words, word_importances = subwords, subword_importances
    word_importances = np.array(word_importances) / np.linalg.norm(
        word_importances)
    
    true_label = true_labels[i]
    
    if interp_class == 0:
        word_importances = -word_importances
        
    rec = VisualizationTextRecord(
        words, 
        word_importances, 
        true_label,                   
        pred_class[0], 
        pred_prob[0],
        interp_class
    )
    
    recs.append(rec)

visualize_text(recs)

## GradShapNLPInterpreter

In [11]:
ig = it.GradShapNLPInterpreter(model_, True)

true_labels = [0, 0, 1]
recs = []

for review in data:
    text = review['text']

    pred_labels, pred_probs, avg_gradients = ig.interpret(
        preprocess_fn(text),
        n_samples=50,
        noise_amount=0.1,
        embedding_name='embedder',
        return_pred=True)
    
    for i in range(avg_gradients.shape[0]):
        subwords = " ".join(jieba.cut(text)).split(' ')
        subword_importances = avg_gradients[i]
        words, word_importances = aggregate_subwords_and_importances(subwords, subword_importances)
        word_importances = np.array(word_importances) / np.linalg.norm(
            word_importances)

        pred_label = pred_labels[i]
        pred_prob = pred_probs[i, pred_label]
        true_label = true_labels[i]
        interp_class = pred_label

        if interp_class == 0:
            word_importances = -word_importances
        recs.append(
            VisualizationTextRecord(words, word_importances, true_label,
                                    pred_label, pred_prob, interp_class)
        )

visualize_text(recs)